### Import modules


In [2]:
import brainpy as bp
import brainpy.math as bm
import numpy as np
import matplotlib.pyplot as plt
import tqdm as notebook_tqdm
bm.set_platform('gpu')


In [ ]:
class WendlingModel(bp.DynamicalSystem):
    def __init__(self, A=5, B=22, G=10, a=0.1, b=0.05, g=0.05, p_mean=90, p_sigma=2, e0=2.5, v0=6.0, r=0.56,  mode = None):
        super().__init__(mode = mode)
        self.a = a
        self.b = b
        self.g = g
        self.p_mean = p_mean
        self.p_sigma = p_sigma
        self.e0 = e0
        self.v0 = v0
        self.r = r
        # this model only supports non-batching mode
        bp.check.is_subclass(self.mode, bm.NonBatchingMode)
        #set paramters that you want to trace (those that will update through time)
        self.A = bm.Variable(A) 
        self.B = bm.Variable(B) 
        self.G = bm.Variable(G) 
        # interneuron connections
        self.C = 135.0  # Base connectivity constant
        self.C1 = 1.0 * self.C   # C1 = C
        self.C2 = 0.8 * self.C# C2 = 0.8*C
        self.C3 = 0.25 * self.C # C3 = 0.25*C
        self.C4 = 0.25 * self.C  # C4 = 0.25*C
        self.C5 = 0.3 * self.C   # C5 = 0.3*C
        self.C6 = 0.1 * self.C   # C6 = 0.1*C
        self.C7 = 0.8 * self.C  # C7 = 0.8*C

        # add states that change through time
        self.y0 = bm.Variable(0.)
        self.y1 = bm.Variable(0.)
        self.y2 = bm.Variable(0.)
        self.y3 = bm.Variable(0.)
        self.y4 = bm.Variable(0.)
        self.y5 = bm.Variable(0.)
        self.y6 = bm.Variable(0.)
        self.y7 = bm.Variable(0.)
        self.y8 = bm.Variable(0.)
        self.y9 = bm.Variable(0.)
        #define integration method
        self.integral = bp.odeint( f=self._ode, method='exp_auto' )
    def _sigmoid(self, v):
        return (2.0 * self.e0) / (1.0 + bm.exp(self.r * (self.v0 - v))) #special sigmoid that NMMs use

    def _ode(self, y, t, p): #although ODE is autonomous, odeint requires t

        y0, y1, y2, y3, y4, y5, y6, y7, y8, y9 = y
        coupling_input = 0 #for single node
        # external input
        p_t = p #assume p is noise
        dy0 = y5
        dy5 = self.A * self.a * (self._sigmoid(y1 - y2 - y3) + coupling_input) - 2.0 * self.a * y5 - self.a * self.a * y0
        
        dy1 = y6
        dy6 = self.A * self.a * (self.C2 * self._sigmoid(self.C1 * y0) + p_t) - 2.0 * self.a * y6 - self.a * self.a * y1
        
        dy2 = y7
        dy7 = self.B * self.b * (self.C4 * self._sigmoid(self.C3 * y0)) - 2.0 * self.b * y7 - self.b * self.b * y2
        
        dy3 = y8
        dy8 = self.G * self.g * (self.C7 * self._sigmoid((self.C5 * y0 - self.C6 * y4))) - 2.0 * self.g * y8 - self.g * self.g * y3
        
        dy4 = y9
        dy9 = self.B * self.b * (self._sigmoid(C3 * y0)) - 2.0 * self.b * y9 - self.b * self.b * y4
        return dy0, dy1, dy2, dy3, dy4, dy5, dy6, dy7, dy8, dy9
    def update(self):
        t = bp.share.load('t')
        dt = bp.share.load('dt')

        # stochastic input (NO sqrt(dt))
        xi = bm.random.randn()
        p_t = self.p_mean + self.p_sigma * xi

        y = bm.stack([
            self.y0, self.y1, self.y2, self.y3, self.y4,
            self.y5, self.y6, self.y7, self.y8, self.y9
        ])

        y_next = self.integral(y, t, p_t, dt=dt)

        self.y0.value = y_next[0]
        self.y1.value = y_next[1]
        self.y2.value = y_next[2]
        self.y3.value = y_next[3]
        self.y4.value = y_next[4]
        self.y5.value = y_next[5]
        self.y6.value = y_next[6]
        self.y7.value = y_next[7]
        self.y8.value = y_next[8]
        self.y9.value = y_next[9]


In [ ]:
with bm.environment(dt=0.1):
    runner = bp.DSRunner(net1, monitors={'E.spike': net1.E.spike, 'I.spike': net1.I.spike})
    runner.run(inputs=bm.ones(1000) * 20.)

bp.visualize.raster_plot(runner.mon['ts'], runner.mon['E.spike'])